In [7]:
from goatools import obo_parser
from zipfile import ZipFile

import os
import gzip
import graco
import shutil
import requests
import numpy as np
import pandas as pd
import networkx as nx
import Bio.UniProt.GOA as GOA

In [8]:
pd.set_option("display.max_columns", 50)

DATA_DIRECTORY = "/home/clusterduck123/Desktop/git/supplements/data"
RAW_DATA_DIRECTORY = f"{DATA_DIRECTORY}/raw-data"
HUMAN_DIRECTORY = f"{DATA_DIRECTORY}/processed-data/organisms/human"
NETWORK_DIRECTORY = f"{HUMAN_DIRECTORY}/networks"
ANNOTATION_DIRECTORY = f"{HUMAN_DIRECTORY}/annotations"

# Downloads

### BioGRID

In [9]:
if not os.path.exists(RAW_DATA_DIRECTORY):
     os.makedirs(directory)

VERSION = "3.5.178"
BioGRID_FILENAME = f"BIOGRID-ORGANISM-{VERSION}.tab2.zip"
BioGRID_URL = f"https://downloads.thebiogrid.org/Download/BioGRID/Release-Archive/BIOGRID-{VERSION}"
BioGRID_FILEPATH = f"{RAW_DATA_DIRECTORY}/{BioGRID_FILENAME}" 

In [ ]:
# Download
r = requests.get(f"{BioGRID_URL}/{BioGRID_FILENAME}", allow_redirects=True)
with open(BioGRID_FILEPATH, 'wb') as f:
    f.write(r.content)

#### Human

In [ ]:
# Unzip Homo Sapiens file
with ZipFile(BioGRID_FILEPATH, 'r') as z:
    BioGRID_sc_FILENAME, = [name for name in z.namelist() if 'sapiens' in name.lower()]
    z.extract(BioGRID_sc_FILENAME, RAW_DATA_DIRECTORY)

### Human annotations (EBI)

In [ ]:
EBI_FILENAME = "goa_human.gaf.gz"
EBI_URL = "http://geneontology.org/gene-associations"
EBI_FILEPATH = f"{RAW_DATA_DIRECTORY}/{EBI_FILENAME}"

In [ ]:
# Download
r = requests.get(f"{EBI_URL}/{EBI_FILENAME}", allow_redirects=True)
with open(EBI_FILEPATH, 'wb') as f:
    f.write(r.content)

### GO

In [ ]:
GO_FILENAME = "go-basic.obo"
GO_URL = "http://purl.obolibrary.org/obo/go"
GO_FILEPATH = f"{RAW_DATA_DIRECTORY}/{GO_FILENAME}"

In [ ]:
# Download
r = requests.get(f"{GO_URL}/{GO_FILENAME}", allow_redirects=True)
with open(GO_FILEPATH, 'wb') as f:
    f.write(r.content)

# BioGRID PPI

## H. Sapiens

### Define interactor universes

In [ ]:
VERSION = "3.5.178"

In [ ]:
# load BioGRID file as dataframe
organism_FILENAME = f"BIOGRID-ORGANISM-Homo_sapiens-{VERSION}.tab2.txt"
organism_FILEPATH = f"{RAW_DATA_DIRECTORY}/{organism_FILENAME}"

# remove entrez ids and scores because of mixed datatypes 
BioGRID_df = pd.read_csv(organism_FILEPATH, delimiter='\t',
                           usecols=[index for index in range(24) if index not in {1,2,18}])

In [ ]:
# BioGRID universe is the collection of every gene known to BioGRID
universe = set(BioGRID_df['BioGRID ID Interactor A']) | \
           set(BioGRID_df['BioGRID ID Interactor B'])

symbol_universe = set(BioGRID_df['Official Symbol Interactor A']) | \
                  set(BioGRID_df['Official Symbol Interactor B'])

In [ ]:
# Filter for (reliable) physical interactions
EXPERIMENTAL_SYSTEM = {'Two-hybrid', 
                       'Affinity Capture-Luminescence',
                       'Affinity Capture-MS', 
                       'Affinity Capture-RNA', 
                       'Affinity Capture-Western'}
EXPERIMENTAL_SYSTEM_TYPE = {'physical'} # redundant because of experimental evidence filtering

physical_interaction_df = BioGRID_df[BioGRID_df['Experimental System'].isin(EXPERIMENTAL_SYSTEM)]
physical_interaction_df = physical_interaction_df[
    physical_interaction_df['Experimental System Type'].isin(EXPERIMENTAL_SYSTEM_TYPE)] # just in case...


physical_universe = set(physical_interaction_df['BioGRID ID Interactor A']) | \
                    set(physical_interaction_df['BioGRID ID Interactor B'])

symbol_physical_universe = set(physical_interaction_df['Official Symbol Interactor A']) | \
                           set(physical_interaction_df['Official Symbol Interactor B'])

### Define PPI

In [8]:
# PPI in dataframe
PPI_df = physical_interaction_df[
    (physical_interaction_df['Organism Interactor A'] == 9606) & \
    (physical_interaction_df['Organism Interactor B'] == 9606)]

PPI_universe = set(PPI_df['Official Symbol Interactor A']) | \
               set(PPI_df['Official Symbol Interactor B'])

In [9]:
# Reduce PPI to simple network
PPI_nx = nx.from_pandas_edgelist(PPI_df,'Official Symbol Interactor A', 'Official Symbol Interactor B')
PPI_nx.remove_edges_from(nx.selfloop_edges(PPI_nx))

### Summary

In [10]:
print("BioGRID universe sizes:")
print("=========================")
print(f"  -       -    : {len(universe)}")
print(f"symbol    -    : {len(symbol_universe)}")
print(f"  -    physical: {len(physical_universe)}")
print(f"symbol physical: {len(symbol_physical_universe)}")
print("-------------------------")
print(f"PPI population size: {PPI_nx.number_of_nodes()}")

BioGRID universe sizes:
  -       -    : 24277
symbol    -    : 23416
  -    physical: 22332
symbol physical: 21799
-------------------------
PPI population size: 17137


### Save

In [11]:
nx.write_edgelist(PPI_nx, f"{PPI_DIRECTORY}/hs_BioGRID.txt", data=False)

# Annotations

## EBI (H. Sapiens)

In [12]:
EBI_FILENAME = "goa_human.gaf.gz"
EBI_FILEPATH = f"{RAW_DATA_DIRECTORY}/{EBI_FILENAME}"

# load SGD gaf-file as dataframe 
with gzip.open(EBI_FILEPATH, 'rt') as gz:
    EBI_df = pd.DataFrame(annotation for annotation in GOA.gafiterator(gz))

In [13]:
# Filter for proteins
lc_protein_gaf_df = EBI_df[EBI_df.DB_Object_Type == 'protein']

# Filter through evidence code
protein_gaf_df = lc_protein_gaf_df[lc_protein_gaf_df['Evidence'].isin(['EXP', 'IDA', 'IPI', 'IMP', 'IGI', 'IEP'])]

# Split into the three GOs
protein_BP_gaf_df = protein_gaf_df[protein_gaf_df['Aspect']=='P']
protein_MF_gaf_df = protein_gaf_df[protein_gaf_df['Aspect']=='F']
protein_CC_gaf_df = protein_gaf_df[protein_gaf_df['Aspect']=='C']

In [14]:
# Get rid of all unnecesarry columns in the GAFs
high_IC_annotations_df = protein_gaf_df[['DB_Object_Symbol', 'GO_ID']].dropna().drop_duplicates()

# Split into the three GOs
high_IC_BP_annotations_df = protein_BP_gaf_df[['DB_Object_Symbol', 'GO_ID']].dropna().drop_duplicates()
high_IC_MF_annotations_df = protein_MF_gaf_df[['DB_Object_Symbol', 'GO_ID']].dropna().drop_duplicates()
high_IC_CC_annotations_df = protein_CC_gaf_df[['DB_Object_Symbol', 'GO_ID']].dropna().drop_duplicates()

### GO DAG extention (Human2GO)

In [15]:
# Load obo and gaf files
GO_FILENAME = "go-basic.obo"
GO_FILEPATH = f"{RAW_DATA_DIRECTORY}/{GO_FILENAME}"

# Create annotations with all upstream terms in dict
go_dag = obo_parser.GODag(GO_FILEPATH)
go2parents = {go_id:{*go_dag[go_id].get_all_parents(), go_id} for go_id in go_dag.keys()}

/home/clusterduck123/Desktop/git/supplements/data/raw_data/go-basic.obo: fmt(1.2) rel(2019-10-07) 47,285 GO Terms


In [16]:
# Create full list/dataframe of annotations
all_annotations_list = [(gene_id, go_term, go_dag[go_term].level) 
                                for _, (gene_id, go_id) in high_IC_annotations_df.iterrows()
                                    for go_term in go2parents[go_id]]
all_annotations_df = pd.DataFrame(
                            all_annotations_list,  
                            columns = ['DB_Object_Symbol', 'GO_ID', 'Level']).drop_duplicates()

# Split into the three GOs
all_BP_annotations_list = [entry for entry in all_annotations_list 
                               if go_dag[entry[1]].namespace == "biological_process"]
all_MF_annotations_list = [entry for entry in all_annotations_list 
                               if go_dag[entry[1]].namespace == "molecular_function"]
all_CC_annotations_list = [entry for entry in all_annotations_list 
                               if go_dag[entry[1]].namespace == "cellular_component"]

all_BP_annotations_df = pd.DataFrame(
                            all_BP_annotations_list,  
                            columns = ['DB_Object_Symbol', 'GO_ID', 'Level']).drop_duplicates()
all_MF_annotations_df = pd.DataFrame(
                            all_MF_annotations_list,  
                            columns = ['DB_Object_Symbol', 'GO_ID', 'Level']).drop_duplicates()
all_CC_annotations_df = pd.DataFrame(
                            all_CC_annotations_list,  
                            columns = ['DB_Object_Symbol', 'GO_ID', 'Level']).drop_duplicates()

### Summary

In [17]:
print("EBI universe sizes:")
print("============================================")
print("Biological Process : " 
    f"{len(set(map(lambda x:x[0],all_BP_annotations_list)))} genes, "
    f"{len(set(map(lambda x:x[1],all_BP_annotations_list)))} GO-IDs ")
print("Molecular Functions: " 
    f"{len(set(map(lambda x:x[0],all_MF_annotations_list)))} genes, "
    f"{len(set(map(lambda x:x[1],all_MF_annotations_list)))} GO-IDs ")
print("Cellular Components: " 
    f"{len(set(map(lambda x:x[0],all_CC_annotations_list)))} genes, "
    f"{len(set(map(lambda x:x[1],all_CC_annotations_list)))}  GO-IDs ")
print('--------------------------------------------')
print("All annotations    : " 
    f"{len(set(map(lambda x:x[0],all_annotations_list)))} genes, "
    f"{len(set(map(lambda x:x[1],all_annotations_list)))} GO-IDs ")

EBI universe sizes:
Biological Process : 8979 genes, 11103 GO-IDs 
Molecular Functions: 12021 genes, 3510 GO-IDs 
Cellular Components: 10067 genes, 1433  GO-IDs 
--------------------------------------------
All annotations    : 13916 genes, 16046 GO-IDs 


In [18]:
all_annotations_df.to_csv(f"{ANNOTATIONS_DIRECTORY}/all_EBI.csv", index=False)

all_BP_annotations_df.to_csv(f"{ANNOTATIONS_DIRECTORY}/BP_EBI.csv", index=False)
all_MF_annotations_df.to_csv(f"{ANNOTATIONS_DIRECTORY}/MF_EBI.csv", index=False)
all_CC_annotations_df.to_csv(f"{ANNOTATIONS_DIRECTORY}/CC_EBI.csv", index=False)

## BioGRID $\cap$ EBI

In [19]:
PPI_nx = nx.read_edgelist(f"{PPI_DIRECTORY}/hs_BioGRID.txt")

all_annotations_df = pd.read_csv(f"{ANNOTATIONS_DIRECTORY}/all_EBI.csv")

all_BP_annotations_df = pd.read_csv(f"{ANNOTATIONS_DIRECTORY}/BP_EBI.csv")
all_MF_annotations_df = pd.read_csv(f"{ANNOTATIONS_DIRECTORY}/MF_EBI.csv")
all_CC_annotations_df = pd.read_csv(f"{ANNOTATIONS_DIRECTORY}/CC_EBI.csv")

In [20]:
PPI_annotations_df = all_annotations_df[all_annotations_df.DB_Object_Symbol.isin(PPI_nx.nodes)]

PPI_BP_annotations_df = all_BP_annotations_df[all_BP_annotations_df.DB_Object_Symbol.isin(PPI_nx.nodes)]
PPI_MF_annotations_df = all_MF_annotations_df[all_MF_annotations_df.DB_Object_Symbol.isin(PPI_nx.nodes)]
PPI_CC_annotations_df = all_CC_annotations_df[all_CC_annotations_df.DB_Object_Symbol.isin(PPI_nx.nodes)]

### Summary

In [21]:
print(r"EBI ∩ BioGRID universe sizes:")
print("============================================")
print("Biological Process : " 
    f"{len(set(PPI_BP_annotations_df.DB_Object_Symbol))} genes, "
    f"{len(set(PPI_BP_annotations_df.GO_ID))} GO-IDs ")
print("Molecular Functions: " 
    f"{len(set(PPI_MF_annotations_df.DB_Object_Symbol))} genes, "
    f"{len(set(PPI_MF_annotations_df.GO_ID))} GO-IDs ")
print("Cellular Components: " 
    f"{len(set(PPI_CC_annotations_df.DB_Object_Symbol))} genes, "
    f"{len(set(PPI_CC_annotations_df.GO_ID))}  GO-IDs ")
print('--------------------------------------------')
print("All annotations    : " 
    f"{len(set(PPI_annotations_df.DB_Object_Symbol))} genes, "
    f"{len(set(PPI_annotations_df.GO_ID))} GO-IDs ")

EBI ∩ BioGRID universe sizes:
Biological Process : 8073 genes, 10843 GO-IDs 
Molecular Functions: 10977 genes, 3397 GO-IDs 
Cellular Components: 9103 genes, 1413  GO-IDs 
--------------------------------------------
All annotations    : 12446 genes, 15653 GO-IDs 


### Save

In [22]:
PPI_annotations_df.to_csv(f"{ANNOTATIONS_DIRECTORY}/all_BioGRID-EBI.csv", index=False)

PPI_BP_annotations_df.to_csv(f"{ANNOTATIONS_DIRECTORY}/BP_BioGRID-EBI.csv", index=False)
PPI_MF_annotations_df.to_csv(f"{ANNOTATIONS_DIRECTORY}/MF_BioGRID-EBI.csv", index=False)
PPI_CC_annotations_df.to_csv(f"{ANNOTATIONS_DIRECTORY}/CC_BioGRID-EBI.csv", index=False)